In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
df = pd.read_csv('data/processed/ukrainian_tweets_processed.csv',index_col=0)
df = df[~df['text_processed'].isna()]
df

/tmp/ipykernel_35747/57971597.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_35747/57971597.py:4: DtypeWarning: Columns (0,1,6,7,8,10,17,18,20,21,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/processed/ukrainian_tweets_processed.csv',index_col=0)


,Unnamed: 0,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,...,original_tweet_username,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,quoted_status_id,quoted_status_userid,quoted_status_username,extractedts,text_processed
Unnamed: 0.1,,,,,,,,,,,,,,,,,,,,,
405,128266,1.319581e+18,orfiarts,"18| 🇺🇦 | Digital artist, gettin my GD degree |...",Україна,106.0,347.0,1219.0,2020-10-23 10:06:43,1648115832357003264.0,...,NaN,1.648096e+18,4.133071e+09,urijah_ghor,False,0.000000e+00,0.000000e+00,NaN,2023-04-18 09:20:46.013550,шо федора комікса карт знаю чесно шось зайшло...
687,128548,8.383213e+17,rogue_corq,@corq's snark account. \n\n#cyber #linux #agit...,Up in ur timeline,2075.0,2046.0,863899.0,2017-03-05 09:32:38,1648117178166804480.0,...,NaN,0.000000e+00,0.000000e+00,NaN,False,0.000000e+00,0.000000e+00,NaN,2023-04-18 02:45:59.411782,польща заборонила ввозити україни десятки найм...
1312,129173,4.047851e+07,iPapaJustify,"Автор ""Брудної Стіни"": \nhttps://t.co/MIx7FfsCRZ","Kyiv, Ukraine",921.0,1048.0,25342.0,2009-05-16 15:07:33,1648120033308991488.0,...,NaN,0.000000e+00,0.000000e+00,NaN,True,1.648053e+18,1.629998e+09,playua,2023-04-18 09:20:45.930491,вкажіть вік опублікувавши гру вийшла
1322,129183,2.207865e+09,ugryn1968,NaN,NaN,771.0,134.0,37590.0,2013-11-21 21:33:58,1648120102225575936.0,...,mil_in_ua,0.000000e+00,0.000000e+00,NaN,False,0.000000e+00,0.000000e+00,NaN,2023-04-18 02:18:16.824588,україна закупила обладнання відбору зберігання...
1395,129256,4.109126e+08,Olesya_Losss,24 • INFP • крабова паличка • average Живчик ...,"Місто на Че, Ukraine",71.0,76.0,3247.0,2011-11-12 18:57:24,1648120472289128448.0,...,NaN,1.648118e+18,1.597235e+18,BaldReview,False,0.000000e+00,0.000000e+00,NaN,2023-04-18 09:20:45.924516,оффтоп взагалі обурює ліниві виблядки кріслах ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63405,63405,8.500441e+08,succub69,мать чулков и иконописи 🌕nice to eat you ♋️Hol...,vin,271.0,4331,25506.0,2012-09-27 22:20:38.000000,1569832165050880000,...,NaN,1.569341e+18,1.496130e+18,coricciv,False,0.000000e+00,0.000000e+00,NaN,2022-09-14 02:45:49.085038,бля
63417,63417,1.510657e+18,abastianszzz,biomedical engineer volunteer photographer люб...,NaN,11.0,13,54.0,2022-04-03 16:33:58.000000,1569832308189925376,...,NaN,1.569832e+18,1.510657e+18,abastianszzz,False,0.000000e+00,0.000000e+00,NaN,2022-09-14 02:45:49.065202,емоційно узагальнила здвиги фазі
63472,63472,1.454512e+18,MarilionNakano,"я люблю цю країну, навіть без кокаїну 💪 Дніпро 🇺🇦","Україна, Дніпро",40.0,17,538.0,2021-10-30 18:48:23.000000,1569832964615290880,...,NaN,1.569485e+18,1.381728e+18,charlotta_cos,False,0.000000e+00,0.000000e+00,NaN,2022-09-14 02:45:49.035899,швидкого мовити одужання


In [2]:
df.shape

(764476, 30)

In [3]:
# !pip install gensim

In [4]:
# from gensim.models import CoherenceModel

# # Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence Score: ', coherence_lda)

In [5]:
# from langdetect import detect
# df['detect'] = detect(str(df['text']))

# for i in df.detect:
#     if i not in ('en'):
#         df.drop(i)

In [6]:
df.text_processed.apply(lambda x : "це" in x.split(' ')).sum()

23

In [7]:
# df.shape

In [10]:
vectorizer = CountVectorizer(
analyzer='word',       
min_df=10,# minimum required occurences of a word 
max_df=0.8,
# stop_words='english',# remove stop words
lowercase=True,# convert all words to lowercase
# token_pattern='{3,}',# num chars > 3
max_features=5000,# max number of unique words
)

data_matrix = vectorizer.fit_transform(df.text_processed)
data_matrix

<764476x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 3033736 stored elements in Compressed Sparse Row format>

In [11]:
lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
)

lda_output = lda_model.fit_transform(data_matrix)

In [12]:
lda_output.shape

(764476, 10)

In [13]:
import pyLDAvis
import pyLDAvis.lda_model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#pyLDAvis extracts information from a fitted LDA topic model to inform an interactive web-based visualization
pyLDAvis.enable_notebook()
pyLDAvis.lda_model.prepare(lda_model, data_matrix, vectorizer, mds='tsne')



/run/media/eingrid/ec26c78b-20bc-47f1-b2d5-33a92d92c9b6/UCU/LA_PROJECT/env/lib/python3.11/site-packages/pyLDAvis/_prepare.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/run/media/eingrid/ec26c78b-20bc-47f1-b2d5-33a92d92c9b6/UCU/LA_PROJECT/env/lib/python3.11/site-packages/pyLDAvis/_prepare.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this wou

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6      132.641937   -8.652854       1        1  14.901528
8       -2.048760  -37.488205       2        1  10.110296
2      -31.072542 -112.760628       3        1  10.034693
1      -21.212898   39.867458       4        1   9.895250
9      -90.656296  -27.427763       5        1   9.399115
4       71.299950  -83.249573       6        1   9.306349
3       91.532906   92.199638       7        1   9.305899
5      -96.065498   71.431717       8        1   9.241400
7       55.269684   17.824486       9        1   9.133910
0        1.433857  120.276192      10        1   8.671560, topic_info=         Term          Freq         Total Category  logprob  loglift
4455  україна  39803.000000  39803.000000  Default  30.0000  30.0000
4823       шо  21703.000000  21703.000000  Default  29.0000  29.0000
1670      зсу  24126.000000  24126.000000  Default  28.0000  28.0000
4448  украина  20645.000000  20645.000000  Default  27.0000  27.0000
943      день  14824.000000  14824.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
132    бачити   1487.302863   1488.206881  Topic10  -5.2027   2.4445
1941   країні   1482.258305   1483.162329  Topic10  -5.2061   2.4445
179       бог   1478.228244   1479.132281  Topic10  -5.2089   2.4445
2004    кінця   1386.672114   1387.576132  Topic10  -5.2728   2.4445
3743    росію   1361.460324   1362.364361  Topic10  -5.2911   2.4445

[345 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
15        1  0.999870  агрессияроссии
24        4  0.999691     азербайджан
49        6  0.999424        анімація
56        8  0.999239           армія
85        4  0.999372         бажання
...     ...       ...             ...
4902      1  0.999454              єс
4917     10  0.999797           ідіть
4972     10  0.999784         історія
4973      4  0.999590         історії
4993      7  0.999557            їсти

[324 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 3, 2, 10, 5, 4, 6, 8, 1])

In [11]:
from sklearn.decomposition import NMF

In [12]:
model = NMF(n_components=10, init='random', random_state=0)

W = model.fit_transform(data_matrix)

In [13]:
len(vectorizer.get_feature_names_out())

5000

In [14]:
W

array([[0.00000000e+00, 1.80145443e-01, 0.00000000e+00, ...,
        2.58232844e-01, 0.00000000e+00, 0.00000000e+00],
       [3.14135306e-03, 0.00000000e+00, 0.00000000e+00, ...,
        1.08549994e-04, 0.00000000e+00, 1.56293425e-01],
       [5.58256046e-04, 1.77404741e-02, 1.12353814e-03, ...,
        1.17891297e-03, 1.05258352e-03, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.60378746e-02, 9.21283786e-03, 0.00000000e+00, ...,
        0.00000000e+00, 2.30777877e-04, 0.00000000e+00],
       [0.00000000e+00, 3.48522747e-01, 4.50622064e-03, ...,
        0.00000000e+00, 7.99347575e-03, 4.13710385e-03]])

In [15]:
components = model.components_
components

array([[0.        , 0.        , 0.        , ..., 0.00112638, 0.00085326,
        0.        ],
       [0.        , 0.02912269, 0.01279153, ..., 0.04648811, 0.00269905,
        0.00170915],
       [0.        , 0.00346054, 0.00019847, ..., 0.01450395, 0.        ,
        0.        ],
       ...,
       [0.        , 0.01677446, 0.00563009, ..., 0.02027916, 0.000151  ,
        0.        ],
       [0.        , 0.00516183, 0.00316473, ..., 0.0164097 , 0.00151841,
        0.        ],
       [0.        , 0.0002516 , 0.        , ..., 0.00262699, 0.        ,
        0.        ]])

In [16]:
import numpy as np
num_words = 15

vocab = np.array(vectorizer.get_feature_names_out())
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in components])
topics = [' '.join(t) for t in topic_words]

In [17]:
topics

['росія путін війнапутін россиїй войнасукраиной країна вторжениероссии війнути агрессияроссии росіянин всу війна україна терорист рф',
 'знати хотіти не людина але робити життя ви думати бачити як ну зробити так працювати',
 'день народження незалежність цитата сьогодні війнути перемога місто юрист адвокат соловїн ростиславкравець останній вітати підписатися',
 '__ ну дякувати так та ви то як думати взагалі мен ___ бути бачити бо',
 'зсу украин війна россиїй войнасукраиной агрессияроссии вторжениероссии всу війнапутін окупант україна война область генштаб отже',
 'російський окупант український військовий область ракета місто росіянин вторгнення на стан доба українська удар обстріл',
 'це країна народ місто зробити українець фото відео бачити слово правда ми точно світа що',
 'шо ну думати казати та робити шоб писати блять так шось нема сказати взагалі русня',
 'мати право країна увага українець якщо жінка ми думка дитина питання значення можливість світ бо',
 'україна слава президент 

In [18]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=data_matrix, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

NameError: name 'id2word' is not defined

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#pyLDAvis extracts information from a fitted LDA topic model to inform an interactive web-based visualization
pyLDAvis.enable_notebook()
pyLDAvis.lda_model.prepare(model, data_matrix, vectorizer, mds='tsne')

